In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
#set directory for importinh my_tools,py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from datetime import datetime
import glob
%matplotlib qt

In [2]:
#set path. Change accoridngly
path_file = '/media/onno/Algemeen/Thesis/'
path = '/media/onno/Algemeen/Thesis/GFS_performace/'

Mean Error

In [ ]:

variablez=['envelope','phasespeed','u_wind']
for variable in variablez:
    print(variable)
    #get file names from imported dictionary
    file_gfs = file_dic[variable]['GFS']
    file_era = file_dic[variable]['ERA5']
    #open netcdf files as xarray datasets
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False).squeeze()
    era = xr.open_dataset(path_file+file_era,decode_times=False).squeeze()
    #change dimension names of ERA5 dataset
    if list(era.coords)[1]=='longitude':
        era=era.rename({'longitude':'lon'})
    if list(era.coords)[2]=='latitude':
        era=era.rename({'latitude':'lat'})

    #Convert dates to standard datetime values
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    era['time']=my_tools.convert_date_era_r(era.time.values)

    #Select only data for 24h intervals
    gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])

    #Calculating daily mean errors 
    

    t1 = datetime.now()
    for i in range(11):
        #Select days for which GEFS data is present and make datasets equally long
        gfs_d = gfs.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-i))],lead=i*24)
        era_d = era.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1+i),'2019-11-30')]) 
        #Check if relevant dates are correct
        assert(gfs_d.time.values[0]==(era_d.time.values[0]-pd.Timedelta(i,'d')))
        #resample data to calculate mean for each quarter in the dataset
        gfs_d_resampled = gfs_d.resample(time='QS-DEC').mean()
        era_d_resampled = era_d.resample(time='QS-DEC').mean()
        #Make boolean array for each day that is present in season       
        #for each dataset calculate mean 
#         gfs_season_mean = gfs_d.mean(dim='time')
#         era_season_mean = era_d.mean(dim='time')
        #Subtract reforecast dataset with reanalysis dataset to get mean error
        diff = gfs_d_resampled - era_d_resampled
        #save mean errors for each lead day to new netcdf file
        diff.to_netcdf(path +'GFS_resampled_mean_error_{}_day_{:02d}.nc'.format(variable,i))
    #Measure duration of calculation

    t2 = datetime.now()
    print(t2-t1)

Root mean square error

In [ ]:
variablez=['envelope','phasespeed','u_wind']
def rmse(diff):
    return np.sqrt((diff ** 2).mean(axis=0))


for variable in variablez:
    print(variable)
    #get file names from imported dictionary
    file_gfs = file_dic[variable]['GFS']
    file_era = file_dic[variable]['ERA5']
    #open netcdf files as xarray datasets
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False).squeeze()
    era = xr.open_dataset(path_file+file_era,decode_times=False).squeeze()
    #change dimension names of ERA5 dataset
    if list(era.coords)[1]=='longitude':
        era=era.rename({'longitude':'lon'})
    if list(era.coords)[2]=='latitude':
        era=era.rename({'latitude':'lat'})

    #Convert dates to standard datetime values
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    era['time']=my_tools.convert_date_era_r(era.time.values)

    #Select only data for 24h intervals
    gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])

    #Calculating daily mean errors 


    t1 = datetime.now()
    for i in range(11):

        #Select days for which GEFS data is present and make datasets equally long
        gfs_d = gfs.sel(time=[i for i in pd.date_range('1984-12-1','2019-11-{}'.format(30-i))],lead=i*24)
        era_d = era.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1+i),'2019-11-30')])
        #Check if relevant dates are correct
        assert(gfs_d.time.values[0]==(era_d.time.values[0]-pd.Timedelta(i,'d')))
        diff_abs = (gfs_d-era_d)**2
        model_rmse = (diff_abs.resample(time='QS-DEC').mean())**0.5
        #resample data to calculate mean for each quarter in the dataset
#         gfs_d_resampled = gfs_d.resample(time='QS-DEC').mean()
#         era_d_resampled = era_d.resample(time='QS-DEC').mean()
        #Make boolean array for each day that is present in season       
        #for each dataset calculate mean 
#         gfs_season_mean = gfs_d.mean(dim='time')
#         era_season_mean = era_d.mean(dim='time')
        #Subtract reforecast dataset with reanalysis dataset to get mean error
        #save mean errors for each lead day to new netcdf file
        model_rmse.to_netcdf(path +'GFS_resampled_RMSE_{}_day_{:02d}_alt.nc'.format(variable,i))
    #Measure duration of calculation

    t2 = datetime.now()
    print(t2-t1)

In [9]:
variablez = ['envelope']
for variable in variablez:
    print(variable)
    #get file names from imported dictionary
    file_gfs = file_dic[variable]['GFS']
    file_era = file_dic[variable]['ERA5']
    #open netcdf files as xarray datasets
    gfs = xr.open_dataset(path_file+file_gfs,decode_times=False).squeeze()
    era = xr.open_dataset(path_file+file_era,decode_times=False).squeeze()
    #change dimension names of ERA5 dataset
    if list(era.coords)[1]=='longitude':
        era=era.rename({'longitude':'lon'})
    if list(era.coords)[2]=='latitude':
        era=era.rename({'latitude':'lat'})

    #Convert dates to standard datetime values
    gfs['time']=my_tools.convert_date_gefs_r(gfs.time.values)
    era['time']=my_tools.convert_date_era_r(era.time.values)

    #Select only data for 24h intervals
    gfs = gfs.sel(lead=gfs.lead.values[[bool(i) for i in gfs.lead%24==0]])
    era = era.sel(time=[i for i in pd.date_range('1984-12-{}'.format(1),'2019-11-30')])
    gfs_sub = gfs.sel(lead=0,lat=slice(75,25),lon=slice(300,358))
    era_sub = era.sel(lat=slice(75,25),lon=slice(300,358))

envelope


In [28]:
from sklearn.metrics import mean_squared_error
# diff = gfs_sub - era_sub 
def rmse(diff):
    return np.sqrt((diff ** 2).mean())
rmse_sklearn = mean_squared_error(gfs_sub.v.values[0,:,:],era_sub.v.values[0,:,:])**0.5

diff_abs = diff**2
rmse_my = (diff_abs.v.values[0,:,:].mean())**0.5
rmse_fun = rmse(diff.v[0,:,:])

print(rmse_sklearn)
print(rmse_my)
print(rmse_fun)



0.9544362288483995
0.9544361663982903
<xarray.DataArray 'v' ()>
array(0.9544362, dtype=float32)
Coordinates:
    time     datetime64[ns] 1984-12-01
    lead     float32 0.0
    level    float64 300.0
